In [39]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
from tqdm import tqdm

[nltk_data] Downloading package punkt to /home/benny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import pytextrank as pyt

In [2]:
df = pd.read_csv('./input/train.csv')

In [3]:
df.head()

,Id,Popularity,Page content
0,0,-1,"<html><head><div class=""article-info""> <span c..."
1,1,1,"<html><head><div class=""article-info""><span cl..."
2,2,1,"<html><head><div class=""article-info""><span cl..."
3,3,-1,"<html><head><div class=""article-info""><span cl..."
4,4,-1,"<html><head><div class=""article-info""><span cl..."


In [5]:
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

In [8]:
df = df.iloc[:100]

In [9]:
from bs4 import BeautifulSoup
df['Page content'] = df['Page content'].apply(preprocessor) # 此步驟約要花五分鐘

In [30]:
from summa import keywords

In [49]:
text = df['Page content'][0]

In [40]:
tqdm.pandas()
df['keywords'] = df['Page content'].progress_apply(lambda x: keywords.keywords(x).replace('\n', ' ')) 

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.91it/s]


In [37]:
keywords.keywords(text).replace('\n', ' ')

'space nasa asteroids asteroid mission said astronauts earth human humanity today astronaut guided tour win proxima originally wednesday piece challenge house ideas goals goal garver launch agency says sabotage unlikely china launches kalil deputy private'

In [46]:
from summa.summarizer import summarize
tqdm.pandas()
df['summary'] = df['Page content'].progress_apply(lambda x: summarize(x, words=20, split=True, language="english")) 

100%|███████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 412.75it/s]


In [50]:
summarize(text)

''

In [47]:
df.head(5)

,Id,Popularity,Page content,keywords,summary
0,0,-1,clara moskowitz for space com 2013 06 19 15 0...,space nasa asteroids asteroid mission said ast...,[]
1,1,1,by christina warren2013 03 28 17 40 55 utcgoog...,warren utcgoogle christina big caveat starting...,[]
2,2,1,by sam laird2014 05 07 19 15 20 utcballin 2014...,fail failed funniest sports fails football mas...,[]
3,3,-1,by sam laird2013 10 11 02 26 50 utccameraperso...,fails fail failed cat cuban video videos youtu...,[]
4,4,-1,by connor finnegan2014 04 17 03 31 43 utcnfl s...,getty images stadium image fans young fan cham...,[]


In [53]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 
def read_article(text):
    article = text.split(". ")
    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    return summarize_text

In [55]:
generate_summary(text)

Indexes of top ranked_sentence order are  []


IndexError: list index out of range

In [56]:
text.split(". ")

[' clara moskowitz for space com 2013 06 19 15 04 30 utc nasa s grand challenge stop asteroids from destroying earth there may be killer asteroids headed for earth and nasa has decided to do something about it the space agency announced a new grand challenge on june 18 to find all dangerous space rocks and figure out how to stop them from destroying our planet the new mission builds on projects already underway at nasa including a plan to capture an asteroid pull it in toward the moon and send astronauts to visit it as part of the grand challenge the agency issued a request for information today aiming to solicit ideas from industry academia and the public on how to improve the asteroid mission plan we re asking for you to think about concepts and different approaches for what we ve described here william gerstenmaier nasa s associate administrator for human explorations and operations said yesterday during a nasa event announcing the initiative we want you to think about other ways of